In [1]:
import json
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata
from api_keys import MyAppToken

from datetime import datetime, date, timedelta
import time

# Set to True if you want "all the info messages"
debug = False;
print(MyAppToken)

OFJvlNn5ANvzdPSrpGWIXxSW5


In [2]:
todaysDate = time.strftime("%m-%d-%Y")
todaysFile = todaysDate + ".csv"

In [3]:
# Seattle datasets:
# https://data.seattle.gov/Public-Safety/Call-Data/33kz-ixgy

# https://data.seattle.gov/Public-Safety/SPD-Crime-Data-2008-Present/tazs-3rd5
crime_data = "tazs-3rd5"
client = Socrata("data.seattle.gov", MyAppToken)

In [65]:
df = pd.DataFrame(
    client.get(
        crime_data, 
        #[Offense Start DateTime] = '2015-01-10T12:00:00' and '2015-01-10T14:00:00', 
        # offense = "Robbery",
        # offense_code = 120,
        # offense_start_datetime = "2015-02-01T00:00:00",
        offense_start_datetime BETWEEN "2015-01-31 00:00:00" AND "2015-02-01 23:59:59",
        limit=10,
        exclude_system_fields=True
    )
)
# client.close()
# results = client.get(crime_data, Offense = 'Robbery', limit=2000)
# df_robbery = pd.DataFrame.from_records(results)

SyntaxError: invalid syntax (<ipython-input-65-384b3bacb0b9>, line 8)

In [61]:
df.head()

,_100_block_address,beat,crime_against_category,group_a_b,latitude,longitude,mcpp,offense,offense_code,offense_end_datetime,offense_id,offense_parent_group,offense_start_datetime,precinct,report_datetime,report_number,sector
0,75XX BLOCK OF JONES AVE NW,J2,PROPERTY,A,47.684084070,-122.386460162,BALLARD NORTH,Theft From Motor Vehicle,23F,2015-02-01 09:50:00,7699084092,LARCENY-THEFT,2015-01-31 17:00:00,N,2015-02-01 11:04:00,2015-900680,J
1,70XX BLOCK OF S 115TH ST,S3,PROPERTY,A,47.499708700,-122.246388536,RAINIER VIEW,Motor Vehicle Theft,240,2015-02-01 01:35:00,7695536997,MOTOR VEHICLE THEFT,2015-01-31 17:00:00,S,2015-02-01 01:39:00,2015-036063,S
2,6XX BLOCK OF NW 89TH ST,J1,PROPERTY,A,47.693531050,-122.364674984,GREENWOOD,Burglary/Breaking & Entering,220,2015-03-15 13:00:00,7682924447,BURGLARY/BREAKING&ENTERING,2015-01-31 17:00:00,N,2015-03-15 13:47:00,2015-085270,J
3,24XX BLOCK OF 2ND AVE,D1,PROPERTY,A,47.615084920,-122.347420341,BELLTOWN,Motor Vehicle Theft,240,2015-02-01 17:00:00,7665241463,MOTOR VEHICLE THEFT,2015-01-31 17:00:00,W,2015-02-19 12:41:00,2015-057244,D


In [19]:
# Drop unneeded columns
# Keep only 'latitude', 'longitude', 'crime_against_category', 'offense', 'offense_code', 'report_datetime'
df.drop([
    'beat', 'group_a_b', 'mcpp'
    , 'offense_end_datetime', 'offense_id', 'report_datetime'
    , 'precinct', 'report_number', 'sector'
    ], axis=1, inplace=True
)

# description = "offense"
# primary_type = "parent"
df.columns = ["category", "latitude", "longitude", "description", "code", "primary_type", "datetime"]
df["city"] = "Seattle";
df["domestic"] = "Unknown"

df['day'] = pd.DatetimeIndex(df['datetime']).day
df['month'] = pd.DatetimeIndex(df['datetime']).month
df['year'] = pd.DatetimeIndex(df['datetime']).year

# Organize: 
final = df[["city", "primary_type", "description", "domestic", "day", "month", "year", "latitude", "longitude"]]

In [21]:
final.head()

,city,primary_type,description,domestic,day,month,year,latitude,longitude
0,Seattle,SEX OFFENSES,Fondling,Unknown,31,1,2015,0E-9,0E-9
1,Seattle,SEX OFFENSES,Rape,Unknown,30,1,2015,0E-9,0E-9
2,Seattle,SEX OFFENSES,Rape,Unknown,16,1,2015,0E-9,0E-9
3,Seattle,SEX OFFENSES,Rape,Unknown,29,1,2015,0E-9,0E-9
4,Seattle,SEX OFFENSES,Sodomy,Unknown,27,1,2015,0E-9,0E-9


In [22]:
df.head()

,category,latitude,longitude,description,code,primary_type,datetime,city,domestic,day,month,year
0,PERSON,0E-9,0E-9,Fondling,11D,SEX OFFENSES,2015-01-31 14:06:00,Seattle,Unknown,31,1,2015
1,PERSON,0E-9,0E-9,Rape,11A,SEX OFFENSES,2015-01-30 23:00:00,Seattle,Unknown,30,1,2015
2,PERSON,0E-9,0E-9,Rape,11A,SEX OFFENSES,2015-01-16 15:00:00,Seattle,Unknown,16,1,2015
3,PERSON,0E-9,0E-9,Rape,11A,SEX OFFENSES,2015-01-29 08:30:00,Seattle,Unknown,29,1,2015
4,PERSON,0E-9,0E-9,Sodomy,11B,SEX OFFENSES,2015-01-27 22:13:00,Seattle,Unknown,27,1,2015


In [6]:
x = df["category"].unique()
sorted(x)

KeyError: 'category'

In [ ]:
# .unique() creates a numpy.ndarray
x = df["group"].unique();
sorted(x)

In [ ]:
df.groupby(["category", "group"]).count()

In [ ]:
aggs = df.groupby(["category", "group", "offense"]).count()
aggs.columns = ["incidents", "delete1", "delete12", "delete13"]
aggs = aggs.reset_index()
aggs.columns # aggs.drop(aggs.columns[["delete1", "delete12", "delete13"]], axis=0)
aggs

In [ ]:
df_robbery.dtypes

In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.seattle.gov", None)
results = client.get(crime_data, limit=2000)
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [ ]:
client = Socrata("data.seattle.org", MyAppToken, None, None )

try:
    results = client.get("tazs-3rd5", limit=10)
                         
except requests.exceptions.Timeout:
  print("Timeout occurred")

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [ ]:
# Example authenticated client (needed for non-public datasets):
client = Socrata("data.seattle.org", MyAppToken, None, None);
# url = "https://data.seattle.gov/resource/tazs-3rd5.json?Offense=Robbery"
url = "tazs-3rd5.json"

# results = client.get(url, Offense = "Burglary", limit = 10)
results = client.get(url, limit = 10)

In [ ]:

df_roberry = pd.DataFrame.from_records(results)
df_robbery

In [ ]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("wg3w-h783", incident_year = 2019, limit=20000)

# Convert to pandas DataFrame
df1 = pd.DataFrame.from_records(results)

df1

In [ ]:
url = "https://data.seattle.gov/resource/tazs-3rd5.json?Offense=Robbery"
df_roberry = pd.read_json(url)

df_roberry = df_roberry.drop(columns = [
    "_100_block_address", "beat", "mcpp", "offense_code"
    , "offense_end_datetime", "offense_id", "precinct"
    , "report_number", "sector", "report_datetime"
    , "group_a_b", "offense_parent_group", "crime_against_category"
])
#df_roberry.columns = ["lat", "long", "offense", "date"]
df_roberry.head()

In [ ]:
url = "https://data.seattle.gov/resource/tazs-3rd5.json?Offense=Burglary"
df_burglary = pd.read_json(url)
df_burglary.head()

In [ ]:
csvpath = "data/SPD_Crime_Data__2008-Present.csv"
df = pd.read_csv(csvpath, low_memory=False)
df.head()

In [ ]:
df.size

In [ ]:
df.Offense.unique()

In [ ]:
# Burglary/Breaking & Entering
# Robbery